## Importar librerías

In [51]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.pipeline import Pipeline
from skl2onnx.common.data_types import FloatTensorType
from skl2onnx import convert_sklearn, to_onnx, update_registered_converter
from skl2onnx.common.shape_calculator import calculate_linear_classifier_output_shapes
from onnxmltools.convert.xgboost.operator_converters.XGBoost import convert_xgboost
from onnxmltools.convert import convert_xgboost as convert_xgboost_booster
import warnings
warnings.filterwarnings('ignore')

In [52]:
# Cargar
tester_files_folder = r"C:\Users\Administrador\AppData\Roaming\MetaQuotes\Terminal\Common\Files"
df_buy = pd.read_csv(os.path.join(tester_files_folder, "buy_dataset.csv"))
df_sell = pd.read_csv(os.path.join(tester_files_folder, "sell_dataset.csv"))
print(f"Buy -> Trades: {df_buy.shape[0]} Features: {df_buy.shape[1]-1}")
print(f"Sell -> Trades: {df_sell.shape[0]} Features: {df_sell.shape[1]-1}")

Buy -> Trades: 5228 Features: 296
Sell -> Trades: 4544 Features: 296


In [53]:
df_buy

,data_0,data_1,data_2,data_3,data_4,data_5,data_6,data_7,data_8,data_9,...,data_287,data_288,data_289,data_290,data_291,data_292,data_293,data_294,data_295,target
0,-1.374962,1.748489,5.753642,-0.001131,0.001106,381.0,-0.001131,0.001051,436.0,-0.001357,...,490.0,0.001540,0.000741,541.0,0.001767,0.000715,402.0,0.750000,0.166667,0.0
1,-0.054624,-0.063995,1.759199,-0.000323,0.000408,55.0,-0.000363,0.000413,56.0,-0.000605,...,42.0,-0.000581,0.000236,154.0,-0.000291,0.000213,77.0,0.375000,0.333333,1.0
2,-0.148687,-0.748389,3.152707,-0.000316,0.000325,136.0,-0.000505,0.000316,43.0,-0.000379,...,59.0,-0.000505,0.000385,104.0,-0.000631,0.000419,96.0,0.833333,0.500000,1.0
3,0.007580,-0.616351,6.195088,-0.000542,0.000721,107.0,-0.001083,0.000687,89.0,-0.001016,...,149.0,0.000339,0.000673,115.0,0.000339,0.000683,99.0,0.583333,0.666667,0.0
4,1.362893,2.607512,2.878492,-0.000216,0.000723,36.0,-0.000216,0.000762,34.0,-0.000576,...,66.0,0.001226,0.000855,54.0,0.000432,0.000866,41.0,0.333333,0.333333,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5223,-0.197248,-1.055359,5.827653,-0.000616,0.000457,88.0,-0.000889,0.000437,76.0,-0.000889,...,97.0,-0.000068,0.000653,159.0,-0.000342,0.000663,153.0,0.208333,0.500000,0.0
5224,-1.007548,0.422517,3.617995,-0.000605,0.000624,97.0,-0.000643,0.000631,49.0,-0.000869,...,87.0,0.000847,0.000431,64.0,0.000794,0.000426,110.0,0.333333,0.166667,1.0
5225,0.201204,-0.314596,6.141643,-0.000405,0.000858,591.0,-0.000809,0.001001,461.0,-0.000971,...,200.0,0.000526,0.001023,206.0,0.000000,0.001035,137.0,0.416667,0.500000,1.0
5226,-0.342032,-0.375671,8.559157,-0.000983,0.001210,334.0,-0.001652,0.001238,266.0,-0.001206,...,272.0,0.002237,0.001109,305.0,0.002596,0.001075,319.0,0.750000,0.500000,0.0


In [54]:
df_sell

,data_0,data_1,data_2,data_3,data_4,data_5,data_6,data_7,data_8,data_9,...,data_287,data_288,data_289,data_290,data_291,data_292,data_293,data_294,data_295,target
0,-0.077589,-0.506383,0.818737,0.000038,0.000066,3.0,0.000113,0.000069,8.0,0.000189,...,8.0,0.000264,0.000100,8.0,0.000189,0.000112,17.0,0.916667,0.666667,1.0
1,0.928965,0.655760,16.707880,0.002920,0.004777,751.0,0.003344,0.004677,777.0,0.009179,...,872.0,0.011153,0.003846,911.0,0.006156,0.003504,759.0,0.708333,0.666667,0.0
2,-0.057978,-0.520862,15.692568,0.001231,0.001886,1023.0,0.003345,0.001924,750.0,0.002904,...,1077.0,-0.000146,0.002307,769.0,-0.002195,0.002326,1127.0,0.875000,0.833333,1.0
3,-0.664748,0.001591,3.473012,0.000214,0.000487,26.0,0.000285,0.000820,20.0,0.000642,...,29.0,0.000714,0.000861,25.0,0.000857,0.000852,33.0,0.166667,0.500000,0.0
4,0.518340,-0.102131,11.611258,0.001967,0.003020,941.0,0.001885,0.003026,1140.0,0.003306,...,1925.0,0.003445,0.002115,1670.0,0.002460,0.002028,1435.0,0.458333,0.333333,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4539,0.077832,-0.505154,15.817121,0.001312,0.001025,132.0,0.002363,0.000947,145.0,0.002100,...,58.0,-0.000852,0.000622,75.0,-0.001441,0.000716,71.0,0.541667,0.333333,1.0
4540,-0.229832,-0.517088,16.914341,0.000928,0.001771,1077.0,0.002872,0.001722,1168.0,0.004721,...,149.0,0.003309,0.001067,154.0,0.002759,0.001041,132.0,0.666667,0.166667,1.0
4541,0.088358,-1.066784,10.265433,0.000994,0.000966,696.0,0.002041,0.000969,264.0,0.002259,...,416.0,0.001691,0.000893,212.0,0.001604,0.000907,290.0,0.666667,0.833333,1.0
4542,0.422460,-0.988101,3.196839,0.000447,0.000707,128.0,0.000511,0.000734,113.0,0.000702,...,196.0,-0.000191,0.000679,147.0,0.000191,0.000651,153.0,0.791667,0.333333,1.0


In [55]:
# Chequear y limpiar
if(df_buy.isna().values.any()):
    df_buy=df_buy.dropna()
if(df_sell.isna().values.any()):
    df_sell=df_sell.dropna()

In [56]:
# Dividir 
# Preparación de los datos de compra
X_buy = df_buy.drop(columns='target').values
y_buy = df_buy['target'].values
X_buy_train = X_buy
y_buy_train = y_buy
# Preparación de los datos de venta
X_sell = df_sell.drop(columns='target').values
y_sell = df_sell['target'].values
X_sell_train = X_sell
y_sell_train = y_sell

In [57]:
param_grid = {
    #'feature_selection__threshold': ['mean', 'median'],
    'xgb__n_estimators': [500],
    'xgb__max_depth': [3],
    'xgb__learning_rate': [0.01],
    'xgb__subsample': [0.6],
    'xgb__colsample_bytree': [0.9],
    'xgb__gamma': [0],
    #'scaler__with_mean': [True, False],  # Hiperparámetro del escalador
    #'scaler__with_std': [True, False]    # Hiperparámetro del escalador
}


In [58]:
# Crear pipe y entrenar
from sklearn.decomposition import PCA
pipe_buy = Pipeline([
    ("scaler", StandardScaler()),
    ('feature_selection', PCA(n_components=0.95)),
    ('xgb', xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss'))
    ])
grid_search_buy = GridSearchCV(
    estimator=pipe_buy,
    param_grid=param_grid,
    scoring='accuracy',
    cv=5,
    verbose=1,
    n_jobs=-1
)
grid_search_buy.fit(X_buy_train, y_buy_train)
model_buy = grid_search_buy.best_estimator_
pipe_sell = Pipeline([
    ("scaler", StandardScaler()),
    ('feature_selection', PCA(n_components=0.95)),
    ('xgb', xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss'))
    ])
grid_search_sell = GridSearchCV(
    estimator=pipe_sell,
    param_grid=param_grid,
    scoring='accuracy',
    cv=5,
    verbose=1,
    n_jobs=-1
)
grid_search_sell.fit(X_sell_train, y_sell_train)
model_sell = grid_search_sell.best_estimator_

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits


In [59]:
grid_search_buy.best_score_

0.5487745524074396

In [60]:
grid_search_sell.best_score_

0.5541358320854112

In [61]:
update_registered_converter(
    xgb.XGBClassifier,
    "XGBClassifier",
    calculate_linear_classifier_output_shapes,
    convert_xgboost,
    options={'nocl': [False], 'zipmap': [True, False, 'columns']}
)

In [62]:
model_buy_onnx = convert_sklearn(
    model_buy,
    'pipeline_buy_xgboost',
    [('input', FloatTensorType([None, X_buy_train.shape[1]]))],
    target_opset={'': 12, 'ai.onnx.ml': 2}
)
model_sell_onnx = convert_sklearn(
    model_sell,
    'pipeline_sell_xgboost',
    [('input', FloatTensorType([None, X_buy_train.shape[1]]))],
    target_opset={'': 12, 'ai.onnx.ml': 2}
)
mql5_files_folder = "C:/Users/Administrador/AppData/Roaming/MetaQuotes/Terminal/6C3C6A11D1C3791DD4DBF45421BF8028/MQL5/Files"
with open(os.path.join(mql5_files_folder, "model_buy.onnx"), 'wb') as f:
    f.write(model_buy_onnx.SerializeToString())
with open(os.path.join(mql5_files_folder, "model_sell.onnx"), 'wb') as f:
    f.write(model_sell_onnx.SerializeToString())